In [3]:
!pip install nltk
!pip install mechanize
!pip install html2text

In [4]:
import re
import mechanize
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
from html2text import html2text

url = 'https://www.uol.com.br/vivabem/'
response = get(url)

html_soup = BeautifulSoup(response.text, 'html.parser')
allNews = html_soup.find_all('li', class_ = 'menu-item')

In [5]:
def encontraTodasAsTags(tag, classT = None):
    return html_soup.find_all(tag, class_ = classT)

def clean_html(html):
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", html.strip())
    cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", "", cleaned)
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
    cleaned = re.sub(r"&nbsp;", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    return cleaned.strip()

titles = []
links = []
page_saude = []

for news in allNews:
    title = news.a.text
    titles.append(title)

    link = news.a['href']
    links.append(link)

In [6]:
test_df = pd.DataFrame({'title': titles,'link': links})
print(test_df)

                 title                                               link
0           Home UOL                              https://www.uol.com.br/
1           Produtos    https://clicklogger.rm.uol.com.br/?prd=11&grp=...
2           Notícias                         https://noticias.uol.com.br/
3             Carros                       https://www.uol.com.br/carros/
4           Economia                         https://economia.uol.com.br/
5              Folha                        https://www.folha.uol.com.br/
6            Esporte                      https://www.uol.com.br/esporte/
7             Splash                       https://www.uol.com.br/splash/
8       TV e Famosos                       https://tvefamosos.uol.com.br/
9           Universa                     https://www.uol.com.br/universa/
10           VivaBem                      https://www.uol.com.br/vivabem/
11              Tilt                         https://www.uol.com.br/tilt/
12              Ecoa                  

In [7]:
linksTexto = []
fullPage = []

for link in links:
    br = mechanize.Browser()
    br.set_handle_robots(False)
    br.addheaders = [('User-agent', 'Firefox')]
    html = br.open(link).read().decode('utf-8')
    cleanhtml = clean_html(html)
    fullPage.append(html2text(cleanhtml))
    response = get(link)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    for link in html_soup.find_all('a'):
        linksTexto.append(link.get('href'))
print(len(fullPage))
print(len(linksTexto))

test_df = pd.DataFrame({'text': fullPage})
print(test_df)
test_df.to_csv('scrap3.csv')

33
13843
                                                 text
0   UOL - Seu universo online Acessar a versão Cel...
1   UOL Assine - Acesso ilimitado ao conteúdo excl...
2   UOL Notícias | Notícias do Dia no Brasil e no ...
3   UOL Carros: notícias, lançamentos e avaliações...
4   UOL Economia: encontre orientações e notícias ...
5   Folha de S.Paulo: Notícias, Imagens, Vídeos e ...
6   UOL Esporte - Acompanhe ao vivo os principais ...
7   Splash: Últimas Notícias e Novidades | Splash ...
8   UOL TV e Famosos: Novelas, séries, BBB, realit...
9   Universa: Seu mundo é o mundo – direitos da mu...
10  VivaBem - Bem-estar, fitness, saúde, alimentaç...
11  Tilt | Conteúdo de Tecnologia do UOL PUBLICIDA...
12  ECOA | Por um mundo melhor PUBLICIDADE Home UO...
13  MOV - UOL Play PUBLICIDADE Home UOL Produtos O...
14  NOSSA: Viagem, gastronomia, casa, moda & lifes...
15  UOL TAB: Repórteres na rua em busca da realida...
16  Start: Games, eSports, Streams e Gameplays | U...
17  UOL Educação: P